In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
import os
import time
import zipfile
import smtplib
from email.message import EmailMessage

# Set the directory where files should be downloaded
download_dir = r"H:\My Drive\emory.moore@pop.belmont.edu 2022-11-02 23 57\Belmont\Senior\Second Semester\Data Science Project\Athlete Readiness Project"  

# Set up Chrome options (optional: headless mode for background execution)
chrome_options = Options()
chrome_options.add_argument("--disable-gpu") 
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

# Set Chrome's download behavior to specify the download folder
prefs = {
    "download.default_directory": download_dir,  
    "download.prompt_for_download": False,  # Automatically download without prompting
    "download.directory_upgrade": True,
}
chrome_options.add_experimental_option("prefs", prefs)

# Path to WebDriver
chrome_driver_path = r"C:\Users\emory\chromedriver-win64\chromedriver.exe"

# Set up WebDriver
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open the website
url = "https://cloud.hawkindynamics.com/teams/bhloOzA4Gj6wcDzPfG3g"
driver.get(url)

# Wait until username field is present
wait = WebDriverWait(driver, 10)


#Sign into website
try:
    # Locate the username field and enter the username
    username_field = wait.until(EC.presence_of_element_located((By.NAME, 'username')))  
    username_field.send_keys("enter username") 

    # Click the button to proceed to the password page
    submit_email_button = wait.until(EC.element_to_be_clickable((By.ID, "login-button")))
    submit_email_button.click()
    print("Clicked 'Submit' after entering username.")

    # Wait for the password field to appear
    password_field = wait.until(EC.presence_of_element_located((By.NAME, 'password')))  
    password_field.send_keys("enter password") 

    # Submit the login
    login_button = wait.until(EC.element_to_be_clickable((By.ID, "login-button")))  
    login_button.click()
    print("Logged in successfully.")

except Exception as e:
    print("Login Failed:", e)


#Click on 'Test Type' button 
try:
    test_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Test Type')]")))
    test_button.click()
    print("Clicked 'Test Type' button.")

except Exception as e:
    print("Failed to click 'Test Type':", e)


#Select Test Type 
try:
    # Click the dropdown to expand it
    test_type_dropdown = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@aria-label='test type selector']")))
    test_type_dropdown.click()
    print("Clicked 'Test Type' dropdown.")

    # Small delay to allow options to render
    time.sleep(1)

    # Wait for the dropdown options to be visible and click the option
    option = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), 'Countermovement Jump')]")))
    option.click()
    print("Clicked 'Countermovement Jump' option.")

except Exception as e:
    print("Error selecting test type:", e)


# Click 'Save Filter' button
try:
    save_filter_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Save Filter']")))
    save_filter_button.click()
    print("Clicked 'Save Filter' button.")

except Exception as e:
    print("Failed to click 'Save Filter':", e)


# Handle cookie banner if present
try:
    cookie_banner = driver.find_element(By.CLASS_NAME, "CookieConsent")
    accept_button = cookie_banner.find_element(By.TAG_NAME, "button")  # Adjust if needed
    accept_button.click()
    print("Closed cookie consent banner.")
    time.sleep(2)  # Allow UI to update

except Exception as e:
    print("No cookie banner found or already accepted.")


# Click 'Export Tests' button
try:
    export_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Export Tests']")))
    export_button.click()
    print("Clicked 'Export Tests' button.")

except Exception as e:
    print("Failed to click 'Export Tests':", e)


# Click 'De-identify Data' Checkbox 
try:
    # Locate the checkbox using its name
    deidentify_checkbox = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@name='Deidentify Data' and @type='checkbox']")))

    # Scroll into view (in case it's off-screen)
    driver.execute_script("arguments[0].scrollIntoView();", deidentify_checkbox)

    # Click using JavaScript
    driver.execute_script("arguments[0].click();", deidentify_checkbox)

    print("Checked 'Deidentify Data' checkbox.")

except Exception as e:
    print("Error interacting with 'Deidentify Data' checkbox:", e)


# Function to check if the file is downloaded
def wait_for_download(folder, timeout=180):
    print(f"Checking for downloaded files in: {folder}")
    start_time = time.time()
    while time.time() - start_time < timeout:
        files = [f for f in os.listdir(folder) if f.endswith(".zip") or f.endswith(".crdownload")]
        if files:
            print(f"Found files: {files}")
            return os.path.join(folder, files[0])
        time.sleep(2)
    return None


# Path to the CSV file to overwrite
new_csv_name = "W_Basket_CMJ.csv"

# Check if the old file exists, and delete it if necessary
old_file_path = os.path.join(download_dir, new_csv_name)
if os.path.exists(old_file_path):
    os.remove(old_file_path)
    print(f"Old file {new_csv_name} deleted.")


# Function to extract and rename CSV from ZIP file
def extract_and_rename_csv(zip_path, new_csv_name):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        csv_files = [f for f in zip_ref.namelist() if f.endswith(".csv")]
        if not csv_files:
            print("No CSV file found in ZIP.")
            return None
        csv_filename = csv_files[0]
        zip_ref.extract(csv_filename, os.path.dirname(zip_path))
        extracted_csv_path = os.path.join(os.path.dirname(zip_path), csv_filename)
        
        new_csv_path = os.path.join(os.path.dirname(zip_path), new_csv_name)
        os.rename(extracted_csv_path, new_csv_path)
        print(f"CSV extracted and renamed to: {new_csv_path}")
        return new_csv_path


# Perform the export process
def export_process(driver):
    downloaded_file = wait_for_download(download_dir)
    if downloaded_file:
        print(f"Downloaded file: {downloaded_file}")
        new_csv_name = "W_Basket_CMJ.csv"
        extracted_csv = extract_and_rename_csv(downloaded_file, new_csv_name)
        if extracted_csv:
            os.remove(downloaded_file)
            print(f"Original ZIP file deleted: {downloaded_file}")
        else:
            print("Error extracting the CSV from the ZIP.")
    else:
        print("No file detected after export.")

export_process(driver)


# Close the driver
driver.quit()


# Send email notification for task scheduler
def send_email_notification(success=True):
    gmail_email = "enter email" 
    app_password = "enter password"  
    recipient_email = "enter email" 

    msg = EmailMessage()
    msg["Subject"] = "Automated Script Execution Notification"
    msg["From"] = gmail_email
    msg["To"] = recipient_email

    if success:
        msg.set_content("The web scraping script ran successfully.")
    else:
        msg.set_content("The web scraping script encountered an error.")

    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(gmail_email, app_password)
        server.send_message(msg)
        server.quit()
        print("Email notification sent successfully.")
    except Exception as e:
        print(f"Failed to send email: {e}")


# Call the send_email_notification function at the end to notify success
send_email_notification(success=True)

Clicked 'Submit' after entering username.
Logged in successfully.
Clicked 'Test Type' button.
Clicked 'Test Type' dropdown.
Clicked 'Countermovement Jump' option.
Clicked 'Save Filter' button.
Closed cookie consent banner.
Clicked 'Export Tests' button.
Checked 'Deidentify Data' checkbox.
Old file W_Basket_CMJ.csv deleted.
Checking for downloaded files in: H:\My Drive\emory.moore@pop.belmont.edu 2022-11-02 23 57\Belmont\Senior\Second Semester\Data Science Project\Athlete Readiness Project
Found files: ['bhloOzA4Gj6wcDzPfG3g-08_22_24-03_19_25.zip']
Downloaded file: H:\My Drive\emory.moore@pop.belmont.edu 2022-11-02 23 57\Belmont\Senior\Second Semester\Data Science Project\Athlete Readiness Project\bhloOzA4Gj6wcDzPfG3g-08_22_24-03_19_25.zip
CSV extracted and renamed to: H:\My Drive\emory.moore@pop.belmont.edu 2022-11-02 23 57\Belmont\Senior\Second Semester\Data Science Project\Athlete Readiness Project\W_Basket_CMJ.csv
Original ZIP file deleted: H:\My Drive\emory.moore@pop.belmont.edu 20